# Steam 멀티플레이어 게임 AppID 맵핑 및 최종 데이터 정제

## 조건
- 데이터
    1. 01-1. steam_data.csv : 크롤링·정제된 스팀 멀티플레이어 게임 목록 (한글 게임명 포함)
    2. Steam_AppList.csv : Steam API(ISteamApps/GetAppList)에서 수집한 전체 앱 목록 (영문 게임명 포함)
- 맵핑 기준
    1. 1차: 게임명(영문 → 소문자 변환) 단순 매칭
    2. 2차: 1차 매핑 실패 항목을 Steam 검색 페이지 크롤링을 통해 AppID 수집
    3. 크롤링으로도 수집이 안 된 항목들은 갯수 확인하여 수동으로 수집하거나 삭제
- 중복/부가상품 제거
    1. DLC, 에디션, 번들, 사운드트랙, 코스튬, 키트, 특정 언어(일본어 타이틀) 등 제거
    2. 부정확한 검색결과(특정 index) 수동 제거
- 장르 필터
    - 멀티플레이·슈팅·PvP·협동·MOBA·히어로 슈팅 등 오버워치와 같은 장르가 포함된 게임만 유지
- 출력 데이터
    - 최종 AppID 포함 CSV: 02-3. steam_data_real.csv

## 절차
- Steam API 호출 → 전체 앱 목록 저장
- 1차 맵핑
    - 01-1. steam_data.csv와 Steam_AppList.csv를 게임명(소문자)기준으로 병합
    <br> → 02-1. merged_steam_data.csv
- 2차 맵핑(크롤링)
    - 1차 매핑 실패 항목 리스트(recrawled) 생성
    - Selenium + BeautifulSoup으로 Steam 스토어 검색 페이지 접근
    - AppID 추출 규칙
        - 일반 검색 결과에서 data-ds-appid 읽기
        - 필터링 경고 페이지에서 링크의 /app/{appid}/ 패턴 추출
    - 결과 저장: 02-2. mapped_appids_final.csv
- AppID 재병합
    - 1차 매핑 실패 항목에 2차 매핑 결과 병합
    - 중복 제거 및 컬럼 재정렬 (appid, 게임명, 출시일, 장르, OS, DLC여부, 평가점수, 사용자 평가 수)
- 데이터 정제
    - 부가상품/불필요 항목 필터링
        - "에디션", "번들", "코스튬", "키트", "soundtrack", "mode", "tracks", "패키지", "—", "–", "~" 등
        - 일본어 타이틀, 불필요 index(drop list) 제거
    - 장르 필터 적용: 멀티플레이·슈팅·MOBA·PvP·협동 등만 남김

### 1. 전체 앱목록 저장

In [2]:
import requests
import pandas as pd

url = "https://api.steampowered.com/ISteamApps/GetAppList/v2/"
response = requests.get(url)
data = response.json()

apps = data["applist"]["apps"]

# DataFrame으로 변환
app_df = pd.DataFrame(apps)
print(app_df.head())  # 상위 5개만 보기

# 전체 저장도 가능
app_df.to_csv("Steam_AppList.csv", index=False, encoding="utf-8-sig")


   appid                   name
0      5       Dedicated Server
1      7           Steam Client
2      8                 winui2
3     10         Counter-Strike
4     20  Team Fortress Classic


### 2. appID와 제목 맵핑

In [ ]:
# appid 목록 (영문 제목 기준)
applist_df = pd.read_csv("Steam_AppList.csv")  # 컬럼: appid, name

# 장르 목록 (크롤링된 한글/영문 제목 기반)
multi_df = pd.read_csv("01. steam_data.csv")  # 컬럼: title_eng, genre 등

# 소문자 처리로 매칭률 높이기
applist_df['name'] = applist_df['name'].str.lower()
multi_df['게임명'] = multi_df['게임명'].str.lower()

# 단순 병합
merged_df = pd.merge(multi_df, applist_df, left_on='게임명', right_on='name', how='left')
merged_df.to_csv("02-1. merged_steam_data.csv", index=False, encoding="utf-8-sig")

# 매핑 성공/실패 확인
print("매핑 성공 수:", merged_df['appid'].notnull().sum())
print("매핑 실패 수:", merged_df['appid'].isnull().sum())

# 매핑 실패 항목 살펴보기
print(merged_df[merged_df['appid'].isnull()][['게임명']])

매핑 성공 수: 21308
매핑 실패 수: 3287
                      게임명
12                마법 주문 팀
14            the crew™ 2
17                오버워치™ 2
22     프로야구스피리츠 2024-2025
25        스노우 브레이크: 포비든 존
...                   ...
24533      crabs and rats
24560       adventure jam
24578  the train arrived!
24583    dispatch control
24591    watertrix online

[3287 rows x 1 columns]


### 3. 맵핑 실패 건 크롤링 후 재맵핑

In [5]:
recrawled = merged_df[merged_df['appid'].isnull()][['게임명']]

In [6]:
recrawled = recrawled['게임명'].tolist()

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
import time
import re
import pandas as pd
import urllib.parse
from bs4 import BeautifulSoup

# === 1. Chrome 설정 ===
options = Options()
options.add_argument("--start-maximized")
options.add_experimental_option("detach", True)
driver = webdriver.Chrome(options=options)

# === 2. 검색 리스트 ===
recrawled 
title_list = []
appid_list = []

# === 3. 게임명 순회 ===
for title in recrawled:
    print(f"\n🔍 검색 중: {title}")
    encoded = urllib.parse.quote(title)
    url = f"https://store.steampowered.com/search/?term={encoded}&supportedlang=koreana&l=koreana"
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source, "html.parser")

    # 1순위: 일반 검색 결과
    result = soup.select_one("#search_resultsRows a.search_result_row")
    if result:
        appid = result.get("data-ds-appid")
        print(f"✅ 일반 결과: {title} → appid {appid}")
        appid_list.append(appid)
        title_list.append(title)
        continue

    # 2순위: 필터링된 결과에서 추출
    filtered_div = soup.select_one("#search_results_filtered_warning_persistent")
    if filtered_div:
        filtered_link = filtered_div.find("a", href=True)
        if filtered_link:
            href = filtered_link["href"]
            match = re.search(r'/app/(\d+)/', href)
            if match:
                appid = match.group(1)
                print(f"🔍 필터링 결과에서 추출됨 → appid {appid}")
                appid_list.append(appid)
                title_list.append(title)
                continue

    # 실패한 경우
    print("❌ appid 추출 실패")
    appid_list.append(None)
    title_list.append(title)

# === 4. 결과 저장 ===
appid_df = pd.DataFrame({
    "게임명": title_list,
    "appid": appid_list
})
appid_df.to_csv("02-2. mapped_appids_final.csv", index=False, encoding="utf-8-sig")
print("\n📁 저장 완료: 02-2. mapped_appids_final.csv")



🔍 검색 중: 마법 주문 팀
✅ 일반 결과: 마법 주문 팀 → appid 2904000

🔍 검색 중: the crew™ 2
✅ 일반 결과: the crew™ 2 → appid 646910

🔍 검색 중: 오버워치™ 2
✅ 일반 결과: 오버워치™ 2 → appid 2357570

🔍 검색 중: 프로야구스피리츠 2024-2025
🔍 필터링 결과에서 추출됨 → appid 2051010

🔍 검색 중: 스노우 브레이크: 포비든 존
✅ 일반 결과: 스노우 브레이크: 포비든 존 → appid 2668080

🔍 검색 중: 소울워커
✅ 일반 결과: 소울워커 → appid 1825750

🔍 검색 중: ghost of tsushima 디렉터스 컷
✅ 일반 결과: ghost of tsushima 디렉터스 컷 → appid 2215430

🔍 검색 중: 인슈라오디드 (enshrouded)
✅ 일반 결과: 인슈라오디드 (enshrouded) → appid 1203620

🔍 검색 중: dune: awakening 듄 ：어웨이크닝
✅ 일반 결과: dune: awakening 듄 ：어웨이크닝 → appid 1172710

🔍 검색 중: 보더랜드 3
✅ 일반 결과: 보더랜드 3 → appid 397540

🔍 검색 중: tom clancy’s the division® 2
✅ 일반 결과: tom clancy’s the division® 2 → appid 2221490

🔍 검색 중: 방 탈출 시뮬레이터
✅ 일반 결과: 방 탈출 시뮬레이터 → appid 1435790

🔍 검색 중: liftoff®: fpv drone racing
✅ 일반 결과: liftoff®: fpv drone racing → appid 1301060

🔍 검색 중: operation: tango
✅ 일반 결과: operation: tango → appid 1335790

🔍 검색 중: far cry® 6
✅ 일반 결과: far cry® 6 → appid 2369390

🔍 검색 중: 루마섬
✅ 일반 결과: 루마섬

In [ ]:
# appid 목록 (영문 제목 기준)
appid_df = pd.read_csv("02-2. mapped_appids_final.csv")

# 아까 1차맵핑한 데이터
multi_df = pd.read_csv("02-1. merged_steam_data.csv")  # 컬럼: title_eng, genre 등

In [90]:
import pandas as pd

# appid가 NaN인 행만 추출
no_appid_df = multi_df[multi_df['appid'].isnull()].copy()

In [91]:
# appid_df에서 게임명 기준으로 병합
mapped = pd.merge(
    no_appid_df,
    appid_df[['게임명', 'appid']],
    on='게임명',
    how='left',
    suffixes=('', '_new')
)

# 기존 appid가 NaN인 경우에만 새 appid로 대체
mapped['appid'] = mapped['appid_new'].combine_first(mapped['appid'])
mapped = mapped.drop(columns=['appid_new'])

# 기존 appid가 있는 행과 합치기
with_appid = multi_df[multi_df['appid'].notnull()]
final_df = pd.concat([with_appid, mapped], ignore_index=True)

final_df
# 필요시 저장
# final_df.to_csv("multi_df_appid_merged.csv", index=False, encoding="utf-8-sig")

,게임명,출시일,장르,OS,평가점수,사용자 평가 수,DLC여부,appid,name
0,football manager 2024,2023년 11월 7일,시뮬레이션 | 스포츠 | 경영 | 축구 | 전략,Windows,평가 점수: 매우 긍정적,"사용자 평가 14,753개",0,2252570.0,football manager 2024
1,ride 5,2023년 8월 24일,레이싱 | 시뮬레이션 | 오토바이 | 스포츠 | 분할 화면,Windows,평가 점수: 매우 긍정적,"사용자 평가 2,092개",0,1650010.0,ride 5
2,broken arrow,2025년 6월 20일,전략 | 전쟁 게임 | 현대 | 실시간 전략 | 실시간 전술,Windows,평가 점수: 복합적,"사용자 평가 12,836개",0,1604270.0,broken arrow
3,guilty gear -strive-,2021년 6월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,평가 점수: 매우 긍정적,"사용자 평가 43,066개",0,1384160.0,guilty gear -strive-
4,magic: the gathering arena,2023년 5월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,평가 점수: 복합적,"사용자 평가 21,004개",0,2141910.0,magic: the gathering arena
...,...,...,...,...,...,...,...,...,...
24590,crabs and rats,출시 예정,액션 | 캐주얼 | 아케이드 | 플랫폼 | 2D 플랫폼,Windows,NaN,NaN,0,1723880,NaN
24591,adventure jam,출시 예정,어드벤처 | 액션 | RPG | 2D 격투 | 액션 어드벤처,Windows,NaN,NaN,0,1782170,NaN
24592,the train arrived!,발표 예정,어드벤처 | RPG | 액션 | 기차 | 퍼즐,Windows,NaN,NaN,0,346110,NaN
24593,dispatch control,발표 예정,시뮬레이션 | 전략 | 캐주얼 | 실시간 전략 | 대전략,Windows,NaN,NaN,0,3010460,NaN


### 4. 최종파일 정제

In [92]:
final_df = final_df.drop(columns=['name'])

In [93]:
final_df = final_df[['appid', '게임명', '출시일', '장르', 'OS','DLC여부', '평가점수', '사용자 평가 수']]

In [94]:
final_df = final_df[~final_df['appid'].astype(str).str.contains(',')]

In [95]:
# 쉼표로 분리해서 첫 번째 값만 남기고 숫자로 변환
final_df['appid'] = final_df['appid'].astype(str).str.split(',').str[0]
final_df['appid'] = pd.to_numeric(final_df['appid'], errors='coerce').astype('Int64')

In [96]:
final_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24592 entries, 0 to 24594
Data columns (total 8 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   appid     24332 non-null  Int64 
 1   게임명       24555 non-null  object
 2   출시일       24450 non-null  object
 3   장르        24591 non-null  object
 4   OS        24590 non-null  object
 5   DLC여부     24592 non-null  int64 
 6   평가점수      13540 non-null  object
 7   사용자 평가 수  13540 non-null  object
dtypes: Int64(1), int64(1), object(6)
memory usage: 1.7+ MB


In [98]:
final_df = final_df[~final_df["게임명"].str.contains(r"에디션|번들|코스튬|키트", flags=re.IGNORECASE, na=False)]

In [99]:
final_df = final_df[~final_df["게임명"].str.contains(r"マーダーミステリーｊ|バトルスピリッツ クロスオーバー|ビビッター　追加ミニゲーム|ダンジョン人狼", flags=re.IGNORECASE, na=False)]

In [101]:
final_df[final_df['appid'].isnull()]

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
21410,<NA>,oddsparks: 자동화 모험,2025년 5월 28일,건설 | 싱글 플레이어 | 협동 | 절차적 생성 | 자동화,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 1,536개"
21417,<NA>,goddess of fate: waifu rpg,2025년 4월 1일,전략 | RPG | 애니메이션 | JRPG | 턴제 전략,Windows,0,평가 점수: 대체로 긍정적,사용자 평가 237개
21425,<NA>,용과 같이0 맹세의 장소,2018년 8월 1일,풍부한 스토리 | 액션 | 비뎀업 | 웅장한 사운드트랙 | 범죄,Windows,0,평가 점수: 압도적으로 긍정적,"사용자 평가 59,046개"
21453,<NA>,슈퍼 테크노스 월드 ~열혈 & 아케이드 클래식 컬렉션~,2025년 4월 24일,액션 | 스포츠 | 아케이드 | 액션 어드벤처 | 2D 플랫폼,Windows,0,평가 점수: 대체로 긍정적,사용자 평가 18개
21485,<NA>,call of duty® 4: modern warfare® (2007),2007년 11월 13일,1인칭 슈팅 | 액션 | 멀티플레이어 | 슈팅 | 싱글 플레이어,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 20,417개"
...,...,...,...,...,...,...,...,...
24538,<NA>,sid meier's civilization® iii complete,2006년 10월 25일,전략 | 턴제 전략 | 턴제 | 클래식 | 4X,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 5,280개"
24539,<NA>,call of duty® (2003),2006년 10월 14일,1인칭 슈팅 | 제2차 세계 대전 | 액션 | 클래식 | 멀티플레이어,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 6,518개"
24545,<NA>,荒野国度 legends of the wild,출시 예정,전략 | 4X | 판타지 | 턴제 전략 | 전쟁 게임,Windows,0,NaN,NaN
24546,<NA>,druid's crown,발표 예정,어드벤처 | RPG | 액션 RPG | 아케이드 | 탄막 슈팅,Windows,0,NaN,NaN


### 4. 누락된 appid(163행) 수동으로 수집

In [ ]:
final_df.to_csv("01-2. steam_data_fianl.csv", index=False, encoding='utf-8-sig')

In [ ]:
final_df = pd.read_csv("01-2. steam_data_final.csv", encoding= 'cp949')
final_df

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
0,2252570,football manager 2024,2023년 11월 07일,시뮬레이션 | 스포츠 | 경영 | 축구 | 전략,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 14,753개"
1,1650010,ride 5,2023년 08월 24일,레이싱 | 시뮬레이션 | 오토바이 | 스포츠 | 분할 화면,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 2,092개"
2,1604270,broken arrow,2025년 06월 20일,전략 | 전쟁 게임 | 현대 | 실시간 전략 | 실시간 전술,Windows,0,평가 점수: 복합적,"사용자 평가 12,836개"
3,1384160,guilty gear -strive-,2021년 06월 11일,격투 | 2D 격투 | 웅장한 사운드트랙 | 애니메이션 | PvP,Windows,0,평가 점수: 매우 긍정적,"사용자 평가 43,066개"
4,2141910,magic: the gathering arena,2023년 05월 24일,전략 | 카드 게임 | 트레이딩 카드 게임 | 무료 플레이 | 덱빌딩,Windows,0,평가 점수: 복합적,"사용자 평가 21,004개"
...,...,...,...,...,...,...,...,...
24373,1723880,crabs and rats,출시 예정,액션 | 캐주얼 | 아케이드 | 플랫폼 | 2D 플랫폼,Windows,0,NaN,NaN
24374,1782170,adventure jam,출시 예정,어드벤처 | 액션 | RPG | 2D 격투 | 액션 어드벤처,Windows,0,NaN,NaN
24375,346110,the train arrived!,발표 예정,어드벤처 | RPG | 액션 | 기차 | 퍼즐,Windows,0,NaN,NaN
24376,3010460,dispatch control,발표 예정,시뮬레이션 | 전략 | 캐주얼 | 실시간 전략 | 대전략,Windows,0,NaN,NaN


### 5. 재정제

In [115]:
final_df = final_df[~final_df["게임명"].str.contains(r"인술 특별 수련 시리즈|soundtrack|\s mode|\s tracks| – |探灵笔记·古墓地宫系列|bundle|의상|edition| — | ~ |패키지", flags=re.IGNORECASE, na=False)]

In [ ]:
final_df[final_df["게임명"].str.contains(r"beat saber", flags=re.IGNORECASE, na=False)]

,appid,게임명,출시일,장르,OS,DLC여부,평가점수,사용자 평가 수
129,620980,beat saber,2019년 05월 22일,VR | 리듬 | 음악 | 모드 가능 | 빠른 진행,Windows,0,평가 점수: 압도적으로 긍정적,"사용자 평가 68,387개"
23014,1725507,beat saber: skrillex & wolfgang gartner ? 'the...,2021년 08월 31일,인디,Windows,0,NaN,NaN
23015,1725506,beat saber: skrillex ? 'scary monsters and nic...,2021년 08월 31일,인디,Windows,0,NaN,NaN
23016,1725502,"beat saber: skrillex, justin bieber & don toli...",2021년 08월 31일,인디,Windows,0,NaN,NaN
23017,1725501,"beat saber: skrillex, starrah & four tet ? 'bu...",2021년 08월 31일,인디,Windows,0,NaN,NaN
23066,1636342,beat saber: the pussycat dolls ? 'don't cha',2021년 05월 27일,인디,Windows,0,NaN,NaN
23067,1636343,"beat saber: lmfao ft. lauren bennett, goonrock...",2021년 05월 27일,인디,Windows,0,평가 점수: 긍정적,사용자 평가 31개
23068,1636345,beat saber: maroon 5 ? 'sugar',2021년 05월 27일,인디,Windows,0,NaN,NaN
23069,1636346,beat saber: gwen stefani ? 'the sweet escape f...,2021년 05월 27일,인디,Windows,0,NaN,NaN


In [118]:
drop_idx = [23014,23015,23016,23017,23066,23067,23068,23069]
final_df = final_df.drop(index=drop_idx)
final_df = final_df.reset_index(drop=True)

In [121]:
final_df = final_df[final_df["장르"].str.contains(r"무료 플레이|MOBA|히어로 슈팅|1인칭 슈팅|3인칭 슈팅|멀티플레이어|1인칭|팀 기반|3인칭|PvP|경쟁|슈팅|액션|협동|공상과학|PvE|온라인 협동|시네마틱|깊은 세계관|캐주얼", flags=re.IGNORECASE, na=False)]

In [126]:
final_df.to_csv("02-3. steam_data_real.csv", index=False, encoding='utf-8-sig')